In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))


In [3]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
latest_block = reorg_mon.get_last_block_live()
latest_block

ExtraDataLengthError: The field extraData is 249 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#proof-of-authority for more details. The full extraData is: HexBytes('0xd78301030683626f7288676f312e32322e35856c696e75780000000000000000f89680f893c0c0c180c102c0c104c105c0c107c106c109c0c108c10cc10dc10ac10fc110c111c112c0c114c0c0c0c0c21713c0c0c116c0c21a0ec0c11fc121c122c123c124c0c0c125c21f28c126c129c12bc0c12ac0c0c2282cc12fc132c133c134c135c136c137c0c0c0c3383b1ac13cc13dc13ec13fc13ac131c0c0c144c0c146c145c148c2221fc149c14bc0c142c0c14fc150c151c09dd9e5196b5b741d9a8b016cf749134285b3279a1efb68bc933f74a9731f732050561d641650af761819ea0be00dc9e07dc2e818864006a8c107996141b5b9b401')

In [ ]:
web3.middleware_onion.clear()

In [ ]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
burn_filt = Filter.create_filter(address=None, event_types=[Pair.events.Burn])  # Listen events from any smart contract

In [ ]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [ ]:
start = start - 1000

In [ ]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=burn_filt)

In [ ]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])

    if(len(arguments) >= 1):
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['amount0'] = math.nan

    if(len(arguments) >= 2):
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(arguments[1])
    else:
        event['details']['amount1'] = math.nan    
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


In [ ]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events